<a href="https://colab.research.google.com/github/YueunKim/Kriss_colab/blob/master/6_Optimization_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

f = torch.tensor([[-1., -3., 4.], [-3., 3., -1.]])
target = torch.tensor([0, 1])
criterion = torch.nn.CrossEntropyLoss()
criterion(f, target)

tensor(2.5141)

In [2]:
f = torch.tensor([[-1., -3., 4.], [-3., 3., -1.]])
target = torch.tensor([0, 1])
model = torch.nn.LogSoftmax(dim=1)
criterion = torch.nn.NLLLoss()
criterion(model(f), target)

tensor(2.5141)

In [3]:
y = torch.tensor([[-10., -10., 10., -5.],
                 [3., 0., 0., 0.,],
                 [1., 2., 3., 4.]])
f = torch.nn.Softmax(1)
f(y)

tensor([[2.0612e-09, 2.0612e-09, 1.0000e+00, 3.0590e-07],
        [8.7005e-01, 4.3317e-02, 4.3317e-02, 4.3317e-02],
        [3.2059e-02, 8.7144e-02, 2.3688e-01, 6.4391e-01]])

In [4]:
for e in range(np_epochs):
    output = model(train_input)
    loss = criterion(ouput, train_target)
    
    model.zero_grad()
    loss.backward()
    with torch.no_grad():
        for p in model.parameters():
            p -= eta * p.grad

NameError: ignored

In [5]:
for e in range(np_epochs):
    output = model(train_input)
    loss = criterion(ouput, train_target)
    
    model.zero_grad()
    loss.backward()
    with torch.no_grad():
        for p in model.parameters():
            p -= eta * p.grad

NameError: ignored

In [6]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size = 5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size = 5)
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size = 3))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size = 2))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
import torch
import torchvision
import torch.nn.functional as F
from google.colab import drive

drive.mount('/content/gdrive')
train_set = torchvision.datasets.MNIST('/content/gdrive/MyDrive/pytorch/dataset/mnist/', train = True, download = True)
train_input = train_set.data.view(-1, 1, 28, 28).float()
train_target = train_set.train_labels

lr, nb_epochs, batch_size = 1e-1, 10, 100

model = Net()

optimizer = torch.optim.SGD(model.parameters(), lr = lr)
criterion = nn.CrossEntropyLoss()

device = 'cuda'
model.to(device)
criterion.to(device)
train_input, train_target = train_input.to(device), train_target.to(device)
mu, std = train_input.mean(), train_input.std()
train_input.sub_(mu).div_(std)

for e in range(nb_epochs):
    for input, target in zip(train_input.split(batch_size),train_target.split(batch_size)):
        output = model(input)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
from torch.autograd import Function 

class KillHead(Function):
    @staticmethod
    def forward(ctx, input, n):
        ctx.n = n
        result = input.clone()
        result[:, 0:ctx.n] = 0
        return result

    @staticmethod
    def backward(ctx, grad_output):
        result = grad_output.clone()
        result[:, 0:ctx.n] = 0
        return result, None
    
killhead = KillHead.apply

In [ ]:
y = torch.empty(3, 8).normal_()
x = torch.empty(y.size()).normal_().requires_grad_()
criterion, eta = nn.MSELoss(), 1e-0

for k in range(5):
    r = killhead(x, 2)
    loss = criterion(r, y)
    if k > 0: 
        x.grad.zero_()
    loss.backward()
    print(k, loss.item())
    with torch.no_grad():
        x.sub_(eta * x.grad)
        

In [ ]:
from torch.autograd import gradcheck

x = torch.empty(10, 20, dtype = torch.float64).uniform_(-1, 1).requires_grad_()

input = (x, 4)

if gradcheck(killhead, input, eps = 1e-6, atol = 1e-4):
    print('All good captain.')
else:
    print('Ouch')

In [ ]:
import torch.nn as nn

def create_model(with_batchnorm, nc = 32, depth = 16):
  modules = []
  modules.append(nn.Linear(2, nc))
  if with_batchnorm: 
    modules.append(nn.BatchNorm1d(nc))
  modules.append(nn.ReLU())

  for d in range(depth):
    modules.append(nn.Linear(nc, nc))
    if with_batchnorm: 
      modules.append(nn.BatchNorm1d(nc))
    modules.append(nn.ReLU())
    
  modules.append(nn.Linear(nc, 2))
  
  return nn.Sequential(*modules)


In [ ]:
with torch.no_grad():
  for p in model.parameters(): 
    p.normal_(0, std)

In [ ]:
import torch

u = torch.tensor([1, 2, 3])
u.device

In [ ]:
v = u.to('cuda') #copy of u
v

In [ ]:
v[0] = 5
u

In [ ]:
w = u.to('cpu') #this is u itself
w

In [ ]:
w[0] = 5
u

In [ ]:
m = torch.empty(10, 10).normal_()
m.device

In [ ]:
x = torch.empty(10, 100).normal_()
q = m@x
q.device

In [ ]:
m = m.to('cuda')
x = x.to('cuda')
q = m@x
q.device

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

  model.to(device)
criterion.to(device)
train_input, train_target = train_input.to(device), train_target.to(device)
test_input, test_target = test_input.to(device), test_target.to(device)


In [ ]:
class Dummy(nn.Module):
  def __init__(self, m):
    super(Dummy, self).__init__()
    self.m = m

  def forward(self, x):
    print('Dummy.formard', x.size(), x.device)
    return self.m(x)

x = torch.empty(50, 10).normal_()
model = Dummy(nn.Linear(10, 5))

print('On CPU')
y = model(x)

x = x.cuda()
model.cuda()

print('On GPU w/o nn.DataParallel')
y = model(x)

print('On GPU w/ nn.DataParallel')
parallel_model = nn.DataParallel(model)
y = parallel_model(x)
